<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

## load_audio
We'll start with a basic utilty to read an audio file.  If it's not at the sample rate we want, we'll automatically resample it.

In [1]:
#|output: asis
#| echo: false
show_doc(load_audio)

---

### load_audio

>      load_audio (filename:str, sr=48000, verbose=True)

this loads an audio file as a torch tensor

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| filename | str |  | name of file to load |
| sr | int | 48000 | sample rate in Hz |
| verbose | bool | True | whether or not to print notices of resampling |
| **Returns** | **tensor** |  |  |

Using the file in `examples/`, let's see how this works:

In [ ]:
audio = load_audio('examples/example.wav')

Resampling examples/example.wav from 44100 Hz to 48000 Hz


In [ ]:
audio = load_audio('examples/example.wav',verbose=False)

## is_silence

Sometimes we'll want to know if a file is "silent", i.e. if its contents are quieter than some threshold.  Here's one simple way to implement that:

In [2]:
#|output: asis
#| echo: false
show_doc(audio_float_to_int)

---

### audio_float_to_int

>      audio_float_to_int (waveform)

converts torch float to numpy int16 (for playback in notebooks)

In [ ]:
print(audio.dtype)
print(audio_float_to_int(audio).dtype)

torch.float32
int16


In [3]:
#|output: asis
#| echo: false
show_doc(is_silence)

---

### is_silence

>      is_silence (audio, thresh=-60)

checks if entire clip is 'silence' below some dB threshold

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| audio |  |  | torch tensor of (multichannel) audio |
| thresh | int | -60 | threshold in dB below which we declare to be silence |

Let's test that with some tests.  If all goes well, the following `assert` statements will all pass uneventfully. 

In [ ]:
x = torch.ones((2,10))
assert not is_silence(1e-3*x) # not silent
assert is_silence(1e-5*x) # silent
assert is_silence(1e-3*x, thresh=-50) # higher thresh

## batch_it_crazy
This is a pretty basic utility for breaking up a long sequence into batches, e.g. for model inference

In [4]:
#|output: asis
#| echo: false
show_doc(batch_it_crazy)

---

### batch_it_crazy

>      batch_it_crazy (x, win_len)

(pun intended) Chop up long sequence into a batch of win_len windows

|    | **Details** |
| -- | ----------- |
| x | a time series as a PyTorch tensor, e.g. stereo or mono audio |
| win_len | length of each "window", i.e. length of each element in new batch |

Testing `batch_it_crazy()` for stereo input: 

In [ ]:
x = torch.ones([2,1000])  # stereo
batch_it_crazy(x, 10).shape

torch.Size([101, 2, 10])

...and for mono: 

In [ ]:
x = torch.ones([1000])   # mono
batch_it_crazy(x, 10).shape

torch.Size([101, 1, 10])

...and yeah, currently that "` 1,`" stays because other parts of the code(s) will be assuming "multichannel" audio. 

## makedir 
The next routine creates a directory if it doesn't already exist.  We'll even let it take a "nested" directory such as `a/b/c/d` and the routine will create any directories in that string.

In [5]:
#|output: asis
#| echo: false
show_doc(makedir)

---

### makedir

>      makedir (path:str)

creates directories where they don't exist

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| path | str | directory or nested set of directories |

## get_audio_filenames
Often we'll want to grab a long list of audio filenames by looking through a directory and all its subdirectories.  We could use something like `glob`, `glob` turns out to be extremely slow when large numbers of files (say, more than 100,000) are involved.  Instead we will use the much faster `os.scandir()`, which was packaged nicely into the following routine in [an answer to a StackOverflow question](https://stackoverflow.com/a/59803793/4259243) from which this code is modified:

In [6]:
#|output: asis
#| echo: false
show_doc(fast_scandir)

---

### fast_scandir

>      fast_scandir (dir:str, ext:list)

very fast `glob` alternative. from https://stackoverflow.com/a/59803793/4259243

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| dir | str | top-level directory at which to begin scanning |
| ext | list | list of allowed file extensions |

Quick test:

In [ ]:
_, files = fast_scandir('.', ['wav','flac','ogg','aiff','aif','mp3'])
files

['./examples/example.wav']

Often, rather than being given a single parent directory, we may be given a list of directories in which to look for files.  The following just called `fast_scandir()` for each of those:

In [7]:
#|output: asis
#| echo: false
show_doc(get_audio_filenames)

---

### get_audio_filenames

>      get_audio_filenames (paths:list)

recursively get a list of audio filenames

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| paths | list | directories in which to search |

Here's a fun trick to show off how fast this is: Run in the user's directory tree:

In [ ]:
path = str(os.path.expanduser("~"))+'/Downloads'
if os.path.exists(path):
    files = get_audio_filenames(path)
    print(f"Found {len(files)} audio files.")
else:
    print("Ok it was just a thought.")

Found 105869 audio files.
